# Media Outlets Activity on Wikipedia aggregated by Media Type

The parameters in the cell below can be adjusted to explore other media outlets and time frames.

### How to explore other media types?
The ***media*** parameter can be use to aggregate media outlets by their type. The column `subcategory` in this [this other notebook](../media.ipynb?autorun=true) show the media outlets that belong each type.

***Alternatively***, you can direcly use the [organizations API](http://mediamonitoring.gesis.org/api/organizations/swagger/), or access it with the [SMM Wrapper](https://pypi.org/project/smm-wrapper/).

In [ ]:
# Parameters: 
media = 'TV'
aggregation = 'week' # aggregation can be by day ('day'), week ('week') or month('month')
from_date='2017-09-01'
to_date='2018-12-31'

## Using API

In [ ]:
import pandas as pd
from smm_wrapper import SMMOrganizations
from wikiwho_wrapper import WikiWho

# Create an instance to the smm wrapper
smm = SMMOrganizations()

# Request the politicians from the API
df = smm.dv.get_organizations()
df = df[(df['category']=='media')]

# Filter the accounts by party, and valid ones (the ones that contain wp_ids)
media_df = df[(df['subcategory'].str.contains(media)) & (df['wp_ids'].notnull())] 
wp_ids = set(wp_id for wp_ids in media_df['wp_ids'] for wp_id in wp_ids)
# query the Social Media Monitoring API
wiki_chobs = pd.concat(smm.dv.wikipedia(wikipedia_page_id=wp_id, from_date=from_date, to_date=to_date, aggregate_by=aggregation) 
               for wp_ids in media_df['wp_ids'] for wp_id in wp_ids)
wiki_chobs = wiki_chobs.groupby('date').agg({'chobs': 'sum'}).reset_index()

#using wikiwho to extract conflicts and revisions
ww = WikiWho(lng='de')
wiki_data = pd.concat(ww.dv.edit_persistence(page_id=wp_id, start=from_date, end=to_date) for wp_ids in media_df['wp_ids'] for wp_id in wp_ids )
wiki_data['undos'] = wiki_data['dels'] + wiki_data['reins']
wiki_data = wiki_data[['year_month','conflict', 'elegibles', 'revisions', 'undos']]
wiki_data['date'] = pd.to_datetime(wiki_data['year_month'])
wiki_data = wiki_data.groupby('date').agg({'conflict': 'sum','elegibles':'sum', 'revisions':'sum','undos':'sum' }).reset_index()
wiki_data['conflict_score'] = wiki_data['conflict'] / wiki_data['elegibles']
wiki_data.fillna(0, inplace=True)

In [ ]:
import requests
appended_views = []
session = requests.Session()
for wp_titles in media_df['wp_titles']:
    for wp_title in wp_titles:
        try:
            
            view_url = 'https://wikimedia.org/api/rest_v1/metrics/pageviews/per-article/de.wikipedia.org/all-access/user/{}/daily/{}/{}'.format(wp_title,from_date.replace('-',''),to_date.replace('-',''))
            view = pd.DataFrame(session.get(url=view_url).json()['items'])
            appended_views.append(view)            
          
        except:
            pass
            
views = pd.concat(appended_views)
views = views[['timestamp', 'views']]
views['timestamp']=pd.to_datetime(views['timestamp'], format='%Y%m%d%H')
if aggregation  == 'week':    
    views = views.groupby([pd.Grouper(key='timestamp', freq='W-SUN')])['views'].sum().reset_index().sort_values('timestamp')
    views['timestamp'] = views['timestamp'] - pd.Timedelta(days=6)
    
elif aggregation  == 'month':    
    views = views.groupby([pd.Grouper(key='timestamp', freq='MS')])['views'].sum().reset_index().sort_values('timestamp')

## Plotting

In [ ]:
import plotly
from plotly import graph_objs as go
plotly.offline.init_notebook_mode(connected=True)

plotly.offline.iplot({
    "data": [go.Scatter(x=wiki_chobs['date'], y=wiki_chobs['chobs'], name='Changes', line_shape='spline'),
          go.Scatter(x=views['timestamp'], y=views['views'], name='Views', yaxis='y2')], 
    "layout": go.Layout( title='Wikipedia Activity', xaxis={'title':'Date'}, yaxis=dict(
        title='Views and Changes'
    ),
    yaxis2=dict(
        title='Views',
        
        overlaying='y',
        side='right'
    ), showlegend=True)
})

plotly.offline.iplot({
    "data": [go.Scatter(x=wiki_data['date'], y=wiki_data['undos'], name='Undos', line_shape='spline'),
            go.Scatter(x=wiki_data['date'], y=wiki_data['conflict_score'], name='Conflict', line_shape='spline', yaxis='y2')], 
    "layout": go.Layout(title='Wikipedia Disagreement', xaxis={'title':''}, yaxis=dict(
        title='Undos and Conflict'
    ),
    yaxis2=dict(
        title='Conflict',
        
        overlaying='y',
        side='right'
    ), showlegend=True)
})
